# 참고
- https://pytorch.org/docs/0.4.0/nn.html
- https://pytorch.org/tutorials/
- https://ratsgo.github.io/machine%20learning/2017/10/12/terms/
- https://github.com/DSKSD/Pytorch_Fast_Campus_2018

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import numpy as np
from collections import OrderedDict

torch.manual_seed(1)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# What is a PyTorch?
- Tensor와 Optimizer, Neural Net등 GPU연산에 최적화된 모듈을 이용하여 빠르게 딥러닝 모델을 구현할 수 있는 프레임워크
Facebook이 밀고 있던 lua기반의 torch를 python버전으로 포팅함.

# > Pytorch Basic
- Tensor
- autograd
- nn
- optim

## 2. autograd
- 딥러닝 프레임워크의 가장 큰 장점은 자동으로 미분을 계산해주는 것임.
- 딥러닝에서 forward 단계에서 autograd은 수행하는 모든 연산을 기억함. 그리고 역전파 단계에서  미분을 계산한.

### 2.1 Tensor
- autograd에서 requires_grad=True로 설정되면 Tensor의 연산은 기록됨. 역전파(backward)연산 후에는 이 변수에 대한 gradient(변화도)는 .grad에 누적됨.
- autorgrad구현에서 Function클래스도 매우 중요한데 Tensor와 Function은 상호 연결되어 있으며 모든 연산 과정을 부호화(encode)하여 순환하지 않는 그래프(acyclic graph)를 생성함. 각 변수는 .grad_fn속성을 갖는데 이는 생성한 Function을 참조하고 있음.(단, 사용자가 만든 Tensor는 예외로, 이 때 grad_fn은 None)임.
- gradient를 계산하기 위해서는, Tensor의 .backward()를 호출하면 됨. 

In [2]:
# tensor를 생성하고 연산을 추적하기 위해 requries_grad=True로 설정함.
x= torch.ones(2,2, requires_grad=True)
print(x)

tensor([[ 1.,  1.],
        [ 1.,  1.]])


In [3]:
print(x.data)

tensor([[ 1.,  1.],
        [ 1.,  1.]])


In [4]:
print(x.grad) # 현재 gradient를 계산하지 않음

None


In [5]:
print(x.grad_fn) # 생성된 텐서임

None


In [6]:
# x에 대해 연산을 수행.
y= x+2
print(y)

tensor([[ 3.,  3.],
        [ 3.,  3.]])


In [7]:
# 연산의 결과로 생성된 것이므로 grad_fn을 가짐.
print(y.grad_fn)

In [8]:
# y에 다른 연산을 수행
z= y*y*3
out= z.mean()
print(z)
print(out)

tensor([[ 27.,  27.],
        [ 27.,  27.]])
tensor(27.)


### 2.2 변화도
- backwar에서 Tensor가 Scalar인 경우는 backward에 인자를 정해줄 필요가 없음.  

In [9]:
print(out)
out.backward()
print(x.grad)

tensor(27.)
tensor([[ 4.5000,  4.5000],
        [ 4.5000,  4.5000]])


$$out= \frac{1}{4}\sum_{i}z_{i}$$  
$$z_{i}= 3(x_{i}+2)^{2}$$  
$$\frac{\partial{out}}{\partial{z_{i}}}=\frac{z_{i}}{4}$$  
$$\frac{\partial{z_{i}}}{\partial{x_{i}}}=6(x_{i}+2)$$  
$$\frac{\partial{out}}{\partial{x_{i}}}=\frac{\partial{out}}{\partial{z_{i}}} * \frac{\partial{z_{i}}}{\partial{x_{i}}} = \frac{z_{i}}{4}*6(x_{i}+2) =\frac{3}{2}(x_{i}+2)$$  
$$\frac{\partial{out}}{\partial{x_{i}}}\mid_{x_{i}=1}= \frac{9}{2}$$  

- 하지만 여러개의 요소를 가질 때는 Tensor의 모양을 gradient의 인자로 지정해줄 필요가 있음.
- 인자로 들어간 Tensor가 값이 1일 때는 순수한 gradient이고 값이 k일 때는 gradient*k가 계산됨

In [10]:
x= torch.ones(2,2, requires_grad=True)
y= 2*x+1
try:
    y.backward()
except Exception as e:
    print(e)

grad can be implicitly created only for scalar outputs


In [ ]:
x= torch.ones(2,2, requires_grad=True)
y= 2*x+1
try:
    y.backward(torch.ones(2,2))
except Exception as e:
    print(e)
print(x.grad)

In [ ]:
x= torch.ones(2,2, requires_grad=True)
y= 2*x+1
try:
    y.backward(torch.ones(2,2)*2)
except Exception as e:
    print(e)
    
print(x.grad)

- 기본적으로 변화도 연산은 그래프 상의 모든 내부 버퍼를 새로쓰기(flush) 떄문에, 그래프의 특정 부분에 대해서 역전판 연산을 2번하고 싶다면, 첫 연산 단계에서 retain_variables=True의 값을 넘겨줘야함.

In [ ]:
x= torch.randn(2,2, requires_grad=True)
y= 2*x+2

z= y*y+2
z= z.sum()
try:
    z.backward() # retain_graph=True로 하면 내부 버퍼가 사라지는 것을 막아줌.

    print(x.grad)

    z.backward()
    print(x.grad)
except Exception as e:
    print(e)

In [ ]:
x= torch.randn(2,2, requires_grad=True)
y= 2*x+2

z= y*y+2
z= z.sum()
try:
    z.backward(retain_graph=True) # retain_graph=True로 하면 내부 버퍼가 사라지는 것을 막아줌.

    print(x.grad)

    z.backward()
    print(x.grad)
except Exception as e:
    print(e)

- with torch.no_grad():로 코드 블럭을 사용하면 autograd가 requires_graph=True인 Tensor들의 연산 기록을 추적을 제외할 수 있음.

In [ ]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad)


## 3. nn Module
- nn.Module는 모든 neural network의 기본 class임.
- nn.Module에는 레이어와 출력을 반환하는 메서드 forward가 포함되어있음.
- 미리 포함된 forward함수에 input을 인자로 forward를 계산하고 Parameters의 backward를 계산할 수 있음. 그리고 parameter.grad를 생성함.

### 3.1 nn.Module로 Neural Network생성하기

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__() # 부모클래스 초기화
        self.linear= nn.Linear(2,2) # nn.Linear부분도 다 nn.Module의 자식클래스
        self.sigmoid= nn.Sigmoid() 
        
    def forward(self, inputs):
        outputs= self.linear(inputs)
        outputs= self.sigmoid(outputs)
        
        return outputs

testnn= NN()

print(testnn)
inputs=torch.FloatTensor([1,2])

### 3.2 생성한 NN에 파라미터 혹은 서브 모듈에 접근
- module class의 parameters메서드는 순차적으로 parameter를 반환하는 generator를 생성

In [ ]:
for param in testnn.parameters():
    print(param)

- naemd_parameters는 순차적으로 parameter name과 parameter를 반환하는 generator를 생성


In [ ]:
for name, param in testnn.named_parameters():
    print('[',name,']', param)

In [ ]:
for child in testnn.children():
    print(child)

### 3.3 forward
- 순차적으로 neural network를 계산함.

In [ ]:
inputs= torch.randn(3,2) # nn.Module은 기본적으로 dim=0는 batch size로 봄.
print(inputs)
print("="*50)
print(testnn(inputs))

### 3.4 nn.Sequential()
- 여러 nn.Module을 순차적으로 처리할 수 있는 클래스를 생성.
- 여러 모듈을 간단하게 관리할 수 있음.

In [ ]:
model= nn.Sequential(nn.Linear(2,2),
                    nn.Sigmoid())
print(model)
print(model(inputs))

- OrderedDict로 이름과 함께 관리할 수 있음.

In [ ]:
model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ])) 
print(model)

## 4. optimizer
### 4.1 Loss
- 최적화를 하기 이전에 Loss를 정의해야함.
- Loss는 output(결과값 or 예측값)과 target(실제값)가 얼마나 차이가 나는지 나타내는 측도(measure)로 목적에 맞게 loss function을 정의해야함.
    - example1. 몸무게를 예측하는 경우 MSELoss를 계삼함
    - examapl2. 꽃의 종류를 예측하는 경우 CrossEntorpy를 계산함
    
- torch.nn 패키지에 Loss를 계산해주는 다양한 loss function들이 있음.
    - L1Loss
    - MSELoss
    - CrossEntorpyLoss

#### 4.1.1 nn.MSELoss
- input과 target에 대한 Mean Sqaured Error를 계산함.
$$ Loss(x,y) = \frac{1}{n}\sum_{i}(pred_{i}-target_{i})^2$$

In [ ]:
loss_function= nn.MSELoss() 
preds= torch.FloatTensor([[1,2],[2,0]])
targets= torch.ones(2,2)
loss= loss_function(preds, targets)
print(loss)

#### 4.1.2 CrossEntropyLoss
- Classification 문제에서 주로 사용되는 loss function으로 두 확률분포 사이의 차이를 재는 측도 중 하나임.
- input과 target에 대한 CrossEntropyLoss를 계산함.
- input은 FloatTensor type으로 클래스 수만큼 길이인 array로 구성됨.
- target은 LongTensor로 정답인 클래스의 index로 구성됨.
- CrossEntropy:
$$ H(Target,Pred)= - \sum_{x}Target(x)\log(Q(x))$$


In [ ]:
loss_function= nn.CrossEntropyLoss()
preds= torch.FloatTensor([[0.7, 1.2, 0.6], [0.3, 1, 0.8]])
targets= torch.LongTensor([1, 2])
loss= loss_function(preds, targets)
print(loss)

## 4.2 optim
- torch.optim 패키지를 이용해 모델의 output(결과값)과 target(실제값)에 차이인 loss를 줄이기 위해 parameter update를 통해 최적화함.  
- 'Gradient Descent'라는 방법을 사용해 neural network의 parameter update를 함.  
$ param : \theta$ , $ learning rate: \alpha$ 라고 하면 다음과 같이 업데이트 함.$$ \theta = \theta - \alpha * \frac{\partial{loss}}{\partial{\theta}}$$
- torch.optim은 다양한 optimizer를 가짐.
    - SGD
    - RMSprop
    - Adam
    
- 참고 사이트  
    http://shuuki4.github.io/deep%20learning/2016/05/20/Gradient-Descent-Algorithm-Overview.html

In [ ]:
inputs= torch.randn(5,3)
targets= torch.randn(5).unsqueeze(1)

# model, loss function, optimizer 인스턴스를 생성
model= nn.Linear(3,1)
loss_function= nn.MSELoss()
optimizer= optim.SGD(model.parameters(), lr=0.1)

preds= model(inputs)
print('[preds]', preds)
loss=loss_function(preds, targets)
print('[loss]',loss)

In [ ]:
print('backward 전')
for p in model.parameters():
    print(p.grad)

# 각 parameter에 대한 loss에 미분값을 계산
loss.backward()

print('backward 후')
for p in model.parameters():
    print(p.grad)

- 전과 후를 비교해보면 각 parameter - grad * learning rate로 업데이트 됨.

In [ ]:
print('optimizer 전')
for p in model.parameters():
    print(p.data)

# optimizer로 최적화함
optimizer.step()

print('optimizer 전')
for p in model.parameters():
    print(p.data)